![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [290]:
import pandas as pd
import numpy as np

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Start coding here...

In [292]:
# Explore finance data and drop null values to have good calculations
brands['brand'].value_counts()
print(brands['brand'].unique())
finance['listing_price'].replace(0, np.nan, inplace=True)
finance.dropna(inplace=True)
finance

[nan 'Adidas' 'Nike']


,product_id,listing_price,sale_price,discount,revenue
1,G27341,75.99,37.99,0.5,1641.17
2,CM0081,9.99,5.99,0.4,398.93
3,B44832,69.99,34.99,0.5,2204.37
4,D98205,79.99,39.99,0.5,5182.70
5,B75586,47.99,19.20,0.6,1555.20
...,...,...,...,...,...
3168,AR4347-600,169.95,135.97,0.0,244.75
3170,CQ0278-200,89.95,71.97,0.0,0.00
3172,BV7757-500,159.95,127.97,0.0,230.35
3176,CI1236-100,159.95,127.97,0.0,230.35


In [293]:
# 1) What is the volume of products and average revenue for Adidas and Nike products?
# Add catagorical values for quartiles.
price_quantile = [i for i in finance['listing_price'].quantile([0.25, 0.5, 0.75, 1])]
price_label = []
for quantile in finance['listing_price']:
    if quantile >= price_quantile[0] and quantile < price_quantile[1]:
        price_label.append('Budget')
    elif quantile >= price_quantile[1] and quantile < price_quantile[2]:
        price_label.append('Average')
    elif quantile >= price_quantile[2] and quantile < price_quantile[3]:
        price_label.append('Expensive')
    elif quantile >= price_quantile[3]:
        price_label.append('Elite')
    else:
        price_label.append('NaN')
finance['price_label'] = price_label
finance['price_label'] = finance['price_label'].astype('category')
finance['price_label'] = finance['price_label'].cat.remove_categories('NaN')
finance
# Join tables, create new columns, and create a subset to compare brands.
adidas_vs_nike = finance.merge(brands, on='product_id', suffixes=('_$','_brand'))
adidas_vs_nike.set_index(['brand','price_label'], inplace=True)
adidas_vs_nike['num_products'] = adidas_vs_nike.groupby(['brand','price_label'])['product_id'].count()
adidas_vs_nike['mean_revenue'] = adidas_vs_nike.groupby(['brand','price_label'])['revenue'].mean()
adidas_vs_nike.reset_index(inplace=True)
adidas_vs_nike = adidas_vs_nike[['brand','price_label','num_products','mean_revenue']]
adidas_vs_nike = adidas_vs_nike.groupby(['brand','price_label']).max().round(2)
adidas_vs_nike.reset_index()


,brand,price_label,num_products,mean_revenue
0,Adidas,Average,658.0,4749.96
1,Adidas,Budget,716.0,3188.61
2,Adidas,Elite,2.0,24299.19
3,Adidas,Expensive,583.0,8256.74
4,Nike,Average,57.0,790.50
5,Nike,Budget,7.0,150.99
6,Nike,Elite,NaN,NaN
7,Nike,Expensive,115.0,1350.09


In [295]:
# 2) Do any differences exist between the word count of a product's description and its mean rating?
# Split product description length into bins of 100 words as description_lenght
bins = [0,100,200,300,400,500,600,700]
labels = ['1 to 100', '101 to 200', '201 to 300', '301 to 400', '401 to 500', '501 to 600', '601 to 700']
info['description'].str.len().max()
info['description_length'] = pd.cut(info['description'].str.len(),bins=bins, labels=labels)
info
# Join info df and reviews df in description_lenghts subset
description_lengths = info.merge(reviews, on='product_id', suffixes=('_info','_review'))
description_lengths.set_index('description_length',inplace=True)
# Calculate average rating and num of reviews for new subset df and create new columns
description_lengths['mean_rating'] = description_lengths.groupby('description_length')['rating'].mean()
description_lengths['num_reviews'] = description_lengths.groupby('description_length')['reviews'].sum()
description_lengths.reset_index(inplace=True)
description_lenghts = description_lengths[['description_length','mean_rating','num_reviews']].groupby('description_length').max().round(2)
description_lenghts.reset_index(inplace=True)
description_lenghts

,description_length,mean_rating,num_reviews
0,1 to 100,2.26,36.0
1,101 to 200,3.19,17719.0
2,201 to 300,3.28,76115.0
3,301 to 400,3.29,28994.0
4,401 to 500,3.35,4984.0
5,501 to 600,3.12,852.0
6,601 to 700,3.65,818.0


In [296]:
# 3) How does the volume of products and median revenue vary between clothing and footwear?
# Search in description to clasify footwear vs clothing
info.dropna(inplace=True)
shoes = info[info['description'].str.contains('shoe*|trainer*|foot*')]
clothes = info[~info['description'].str.contains('shoe*|trainer*|foot*')]
# Join shoes and calculate num of shoes and median shoes revenue
shoes_stats = shoes.merge(finance, on='product_id', how='inner')
shoes_median = shoes_stats['revenue'].median()
num_of_shoes = shoes_stats['product_name'].count()
# Join clothes and calculate num of clothes and median clothes revenue
clothes_stats = clothes.merge(finance, on='product_id', how='inner')
clothes_median = clothes_stats['revenue'].median()
num_of_clothes = clothes_stats['product_name'].count()
# Create new DataFrame with stats
product_types = pd.DataFrame({'num_clothing_products':num_of_clothes,'median_clothing_revenue':clothes_median,'num_footwear_products':num_of_shoes,'median_footwear_revenue':shoes_median},index=[0])
product_types




,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,271,1382.08,2495,3249.29
